# Stockage des données
---
Dans cette partie nous nous chargerons de :
- stocké les données nettoyer dans PostgreS
- stocké les données nettoyer dans MongoDB
- Stocké les données brute dans un datalake

Voici les bibliothéques utilisé:

## Bibliothèques et Librairies Utiles

Nous utilisons les bibliothèques suivantes pour interagir avec les données et les bases de données :

1. **`pandas`** :  
   Utilisé pour la manipulation et l'analyse de données sous forme de tableaux (DataFrames). Il permet de lire, écrire, et effectuer des transformations sur des ensembles de données en mémoire.

2. **`psycopg2`** :  
   Une bibliothèque permettant de se connecter et d'interagir avec une base de données PostgreSQL en Python. Elle offre des méthodes pour exécuter des requêtes SQL, récupérer des résultats et gérer des transactions.

3. **`pymongo`** :  
   Permet d'interagir avec MongoDB, une base de données NoSQL. `MongoClient` est l'objet principal pour se connecter à MongoDB, tandis que les fonctions de la bibliothèque permettent de manipuler les collections de documents dans la base de données.

In [ ]:
import pandas as pd
import psycopg2
from pymongo import MongoClient


In [23]:
DB_NAME = "meteo_db"
TABLE_NAME = "meteo_data"

In [24]:
dataset_path = "Nasa_POWER_dataset_cleaned.csv"
df_cleaned = pd.read_csv(dataset_path)
df_cleaned.head(5)

,date,temperature_humide,direction_vent,temperature_air,pression,humidite_relative,composante_est_ouest_vent,temperature_point_rosee,humidite_specifique,intensite_vent,vitesse_vent,precipitations_corrigees
0,2022-01-01,21.41,57.7,25.52,101.26,60.78,-4.70,17.30,12.26,5.74,-2.97,0.00
1,2022-01-02,21.14,49.3,25.01,101.29,62.47,-4.53,17.27,12.23,6.30,-3.89,0.00
2,2022-01-03,20.66,43.1,24.54,101.21,62.79,-3.29,16.78,11.86,5.01,-3.51,0.00
3,2022-01-04,20.69,42.8,24.44,101.11,64.11,-2.41,16.95,12.04,3.73,-2.61,0.00
4,2022-01-05,20.70,52.0,24.35,101.20,64.04,-3.58,17.04,12.08,4.82,-2.80,0.01


In [25]:
dataset_path = "Nasa_POWER_dataset.csv"
df_brute = pd.read_csv(dataset_path)
df_brute.head(5)

,Date,T2MWET,WD10M,T2M,PS,RH2M,U10M,T2MDEW,QV2M,WS10M,V10M,PRECTOTCORR
0,20220101,21.41,57.7,25.52,101.26,60.78,-4.70,17.30,12.26,5.74,-2.97,0.00
1,20220102,21.14,49.3,25.01,101.29,62.47,-4.53,17.27,12.23,6.30,-3.89,0.00
2,20220103,20.66,43.1,24.54,101.21,62.79,-3.29,16.78,11.86,5.01,-3.51,0.00
3,20220104,20.69,42.8,24.44,101.11,64.11,-2.41,16.95,12.04,3.73,-2.61,0.00
4,20220105,20.70,52.0,24.35,101.20,64.04,-3.58,17.04,12.08,4.82,-2.80,0.01


### Insertion sur postgres 

In [27]:
POSTGRES_USER = "postgres"
POSTGRES_PASSWORD = "passer"
POSTGRES_HOST = "localhost"

Vérifions que la BD existe sinon la créer

In [28]:
#On utilise la BD par défaut pour créer notre BD au besoin
conn = psycopg2.connect(dbname="postgres", user=POSTGRES_USER, password=POSTGRES_PASSWORD, host=POSTGRES_HOST)
conn.autocommit = True
cur = conn.cursor()

cur.execute(f"SELECT 1 FROM pg_database WHERE datname = '{DB_NAME}';")
exists = cur.fetchone()
if not exists:
    cur.execute(f"CREATE DATABASE {DB_NAME};")
cur.close()
conn.close()

Insérons les données dans la BD Pstgres

In [30]:
conn = psycopg2.connect(dbname=DB_NAME, user=POSTGRES_USER, password=POSTGRES_PASSWORD, host=POSTGRES_HOST)
cur = conn.cursor()

# Création de la table si elle n'existe pas
cur.execute(f"""
    CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
        id SERIAL PRIMARY KEY,
        date DATE,
        direction_vent FLOAT, temperature_humide FLOAT, pression FLOAT, humidite_specifique FLOAT, temperature_point_rosee FLOAT,
        vitesse_vent FLOAT, intensite_vent FLOAT, temperature_air FLOAT, humidite_relative FLOAT, composante_est_ouest_vent FLOAT, precipitations_corrigees FLOAT
    );
""")

for _, row in df_cleaned.iterrows():
    cur.execute(f"""
        INSERT INTO {TABLE_NAME} (date, direction_vent, temperature_humide,pression,humidite_specifique,temperature_point_rosee,vitesse_vent,intensite_vent,temperature_air,humidite_relative,composante_est_ouest_vent,precipitations_corrigees) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, tuple(row))

conn.commit()
cur.close()
conn.close()

Insérons les données dans mongoDB

In [31]:
MONGO_URI = "mongodb://localhost:27017/"
mongo_client = MongoClient(MONGO_URI)
mongo_db = mongo_client[DB_NAME]
mongo_collection = mongo_db[TABLE_NAME]
mongo_collection.insert_many(df_cleaned.to_dict(orient="records"))


InsertManyResult([ObjectId('67e880ce6cacb9c23634f33e'), ObjectId('67e880ce6cacb9c23634f33f'), ObjectId('67e880ce6cacb9c23634f340'), ObjectId('67e880ce6cacb9c23634f341'), ObjectId('67e880ce6cacb9c23634f342'), ObjectId('67e880ce6cacb9c23634f343'), ObjectId('67e880ce6cacb9c23634f344'), ObjectId('67e880ce6cacb9c23634f345'), ObjectId('67e880ce6cacb9c23634f346'), ObjectId('67e880ce6cacb9c23634f347'), ObjectId('67e880ce6cacb9c23634f348'), ObjectId('67e880ce6cacb9c23634f349'), ObjectId('67e880ce6cacb9c23634f34a'), ObjectId('67e880ce6cacb9c23634f34b'), ObjectId('67e880ce6cacb9c23634f34c'), ObjectId('67e880ce6cacb9c23634f34d'), ObjectId('67e880ce6cacb9c23634f34e'), ObjectId('67e880ce6cacb9c23634f34f'), ObjectId('67e880ce6cacb9c23634f350'), ObjectId('67e880ce6cacb9c23634f351'), ObjectId('67e880ce6cacb9c23634f352'), ObjectId('67e880ce6cacb9c23634f353'), ObjectId('67e880ce6cacb9c23634f354'), ObjectId('67e880ce6cacb9c23634f355'), ObjectId('67e880ce6cacb9c23634f356'), ObjectId('67e880ce6cacb9c23634f3